In [1]:
import pandas as pd
from glob import glob
import numpy as np
from tqdm import tqdm

In [6]:
groups = {}
for group in tqdm(["CN","MCI","AD"],"Opening faster"):
    groups[group]=np.load(open(f"Data/{group}.npy","br"))
    print(len(groups[group]))

Opening faster:  33%|███▎      | 1/3 [00:16<00:33, 16.98s/it]

1326


Opening faster:  67%|██████▋   | 2/3 [00:23<00:10, 10.58s/it]

461


Opening faster: 100%|██████████| 3/3 [00:25<00:00,  8.49s/it]

213


In [ ]:
all_subjects = []
all_images = []
# patient_groups = {}
for group in ["CN","MCI","AD"]:
    files = glob(f"Data/Pre-processed/{group}/*.nii.gz")
    with open(f"Data/{group}.npy","br") as f:
        images = np.load(f)
    assert len(files) == len(images) # Make sure we're not smoked
    all_images.extend(images)
    # patient_groups[group] = patients, images
    'Data/Pre-processed/CN\\1000_sub-ADNI941S6546_ses-M048_T1w2_norm.nii.gz'
    def extract_sub(fp):
        sub = fp[fp.index("S"):]
        return sub[:sub[3:].index("_")+3].replace("_","")
    subjects = [extract_sub(fp) for fp in files]
    all_subjects.extend(subjects)

all_images = np.stack(all_images)
print(all_images.shape)

all_subjects = pd.DataFrame(all_subjects, columns=["subject"])
all_subjects.reset_index(inplace=True)
all_subjects.rename(columns={"index":"imageid"},inplace=True)

all_subjects["Group"] = 0
all_subjects.loc[1326:1326+461,"Group"] = 1
all_subjects.loc[1326+461:1326+461+213,"Group"] = 2

all_subjects

(2000, 80, 96, 80)


,imageid,subject
0,0,S6546
1,1,S6570
2,2,S6570
3,3,S6574
4,4,S6574
...,...,...
1995,1995,S0300
1996,1996,S0916
1997,1997,S1262
1998,1998,S0404


In [176]:
adverse_events = pd.read_csv("Data/Individual level/All_Subjects_ADVERSE_11Mar2025.csv")
adverse_events["subject"] = adverse_events["PTID"].str.split("_")
adverse_events["subject"] = adverse_events["subject"].str[1]+adverse_events["subject"].str[2]
A=set(adverse_events["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
adv_events = adverse_events[["subject","AERELAD","AERELCM","AERELFLRBTBN","AERELFLRBPR","AEHIMG","AERELTAU","AERELNAV","AERELMK","AERELPI","AEHLUMB","AERELCOVID","AERELPAN","AERELATESP","AEHCMEDS","AESERIOUS"]].fillna(0)
for col in adv_events:
    if col =="subject": continue
    col_ = adv_events[col]
    adv_events.loc[col_ > 0, col] = 5 - col_
adv_events

1120 378 904 594


,subject,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,AERELNAV,AERELMK,AERELPI,AEHLUMB,AERELCOVID,AERELPAN,AERELATESP,AEHCMEDS,AESERIOUS
0,S2245,0.0,0.0,0,0,0,0,0.0,0.0,0.0,4,0.0,0.0,0,4.0,0
1,S6041,0.0,0.0,0,0,0,0,0.0,0.0,0.0,2,0.0,0.0,0,0.0,0
2,S6053,0.0,0.0,0,1,1,1,0.0,0.0,0.0,3,0.0,0.0,1,4.0,0
3,S6005,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0
4,S2239,0.0,2.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,S7011,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0
3421,S10031,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,4.0,0
3422,S10031,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0
3423,S6288,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,4.0,0


In [177]:
num_events = all_subjects.groupby(by="subject").size().reset_index().rename(columns={0:"num_events"})
all_subjects = all_subjects.merge(num_events,how="left").fillna(0)
all_subjects = all_subjects.merge(adv_events.groupby(by="subject").sum().reset_index(),how="left").fillna(0)
all_subjects

,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,AERELNAV,AERELMK,AERELPI,AEHLUMB,AERELCOVID,AERELPAN,AERELATESP,AEHCMEDS,AESERIOUS
0,0,S6546,0,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0
1,1,S6570,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,S6570,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,S6574,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0
4,4,S6574,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,1996,S0916,2,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,1997,S1262,2,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,1998,S0404,2,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
backmeds = pd.read_csv("Data/Individual level/All_Subjects_BACKMEDS_11Mar2025.csv")
backmeds["subject"] = backmeds["PTID"].str.split("_")
backmeds["subject"] = backmeds["subject"].str[1]+backmeds["subject"].str[2]

backmeds = backmeds[["subject","KEYMED"]]
med_vocab = list(map(str,list(range(8))))
backmeds_ = []
for i, row in backmeds.iterrows():
    data = {"subject":row["subject"]}
    for word in med_vocab:
        data[word] = int(word in str(row["KEYMED"]))
    backmeds_.append(data)
backmeds_ = pd.DataFrame(backmeds_, columns=["subject","No med",
"Aricept",
"Cognex",
"Exelon",
"Namenda",
"Razadyne",
"Anti-depressant",
"Other behavioral med"]).fillna(0)


A=set(backmeds_["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))

all_subjects=all_subjects.merge(backmeds_.groupby(by="subject").sum().reset_index(),how="left").fillna(0)
all_subjects


3339 538 3283 594


,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,AEHCMEDS,AESERIOUS,No med,Aricept,Cognex,Exelon,Namenda,Razadyne,Anti-depressant,Other behavioral med
0,0,S6546,0,4,0.0,1.0,0.0,0.0,0.0,0.0,...,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,S6570,0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,S6570,0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,S6574,0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,S6574,0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,1996,S0916,2,18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,1997,S1262,2,14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,1998,S0404,2,11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [179]:
neuropath = pd.read_csv("Data/Individual level/All_Subjects_NEUROPATH_11Mar2025.csv")
neuropath["subject"] = neuropath["PTID"].str.split("_")
neuropath["subject"] = neuropath["subject"].str[1]+neuropath["subject"].str[2]
neuropath = neuropath[["subject","NPWBRWT","NPGRCCA","NPGRLA","NPGRHA","NPGRSNH","NPGRLCH","NPAVAS","NPTAN","NPABAN","NPASAN","NPTDPAN","NPHISMB","NPHISG","NPHISSS","NPHIST","NPHISO","NPTHAL","NPBRAAK","NPNEUR","NPADNC","NPDIFF","NPAMY","NPINF","NPHEMO","NPOLD","NPARTER","NPWMR","NPPATH","NPNEC","NPPATHO","NPLBOD","NPNLOSS","NPHIPSCL","NPTDPA","NPTDPB","NPTDPC","NPTDPD","NPTDPE","NPFTDTAU","NPPICK","NPPDXS"]]

A=set(neuropath["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))

# Due to the very limited amount of subjects who have been autopsied in our imaging data, we dont need to merge. not enough data.
# all_subjects=all_subjects.merge(neuropath,how="left")
# all_subjects

692 12 110 594


In [180]:
reccmed = pd.read_csv("Data/Individual level/All_Subjects_RECCMEDS_11Mar2025.csv", dtype="str")
reccmed["subject"] = reccmed["PTID"].str.split("_")
reccmed["subject"] = reccmed["subject"].str[1]+reccmed["subject"].str[2]

A=set(reccmed["subject"])
B=set(all_subjects["subject"])
# print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))

reccmed = reccmed[["subject","CMMED","CMFREQNC","CMREASON", "CMBGN","CMEND","update_stamp"]]
reccmed["CMMED"] = reccmed["CMMED"].str.lower().str.replace("-"," ")
for suffix in [" formula", " oil"," sltb"," la"]:
    reccmed["CMMED"] = reccmed["CMMED"].str.removesuffix(suffix)
for overwrite in ["aspirin","oxybutynin","calcium","garlic","omega 3","probiotic","lactase","acetyl","vitamin b","vitamin c","ascorbic acid",
                  "tylenol","glucosamine","multivitamin","triamter","elidel","nephrovite","ammonium lactate","acetaminophen"]:
    mask = reccmed["CMMED"].str.contains(overwrite).fillna(0).astype("bool")
    reccmed.loc[mask, "CMMED"] = overwrite
# reccmed = reccmed[~pd.isna(reccmed["CMMED"])]
vocab = set(reccmed["CMMED"])
print(len(vocab))
vocab

3442


{'evergreen pearls',
 'mvi (centrum silver)',
 'prostate health',
 'amoxicillan',
 'grapevine',
 'fusidic acid 10mg/g',
 'ectosone regular',
 'clopidogrel bisulfate',
 'didrocal',
 'mineral',
 'flonase',
 'bioflex',
 'draggerthaken',
 'vesicar',
 'beano',
 'l argine',
 'ultimate bone',
 'muscle relaxants',
 'docusate sodium',
 'flut shot',
 'reclasp',
 'monopril',
 'calcium',
 'zitia',
 'refresh',
 'ferrous sulphate',
 'dyazide',
 'hydroxydine',
 'corison injection',
 'fexofenidine',
 'rapaflo',
 'coprofloxacin',
 'flu shot',
 'kirkland acid controller',
 'lipitrol',
 'haldol',
 'bisopropolol/hctz',
 'arjuna (cardiac tonic)',
 'gnc ester c 500',
 'fosinopril',
 'methylprednisolone sodium succinate',
 'azithropine',
 'fesol',
 'gemfibrazol',
 'long acting nitro isosorbid',
 'nitrofarantoin macrocrystal',
 'diltiazem xr',
 'foxamax',
 'clindamyacim',
 'botox injections',
 'nicorette gum',
 'wellbutrin sr',
 'amoxicillin/ clavulanate (augmentin)',
 'high b complex',
 'ca/d',
 'standard pr

In [181]:
common_meds_vocab = reccmed["CMMED"].value_counts().head(30).keys().values

one_hot_df = pd.get_dummies(reccmed["CMMED"])[common_meds_vocab].astype("int")
one_hot_df.columns = ["med_"+col for col in one_hot_df.columns]
one_hot_df.rename(columns={"med_ 4":"med_unknown"},inplace=True)

reccmed_ = pd.concat([reccmed[["subject","CMREASON","CMFREQNC"]], one_hot_df], axis=1)
reccmed_

,subject,CMREASON,CMFREQNC,med_unknown,med_aspirin,med_multivitamin,med_calcium,med_aricept,med_namenda,med_fish,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
0,S0002,NaN,NaN,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72563,S10586,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72564,S10586,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72565,S10586,2,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72566,S10586,2,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [182]:
all_meds_hist = reccmed_.groupby(by="subject").sum()
all_subjects = all_subjects.merge(all_meds_hist.drop(columns=["CMREASON","CMFREQNC"]).reset_index(),how="left")
# all_subjects = all_subjects.merge(all_meds_hist[["CMREASON","CMFREQNC"]].reset_index(),how="left")
all_subjects

,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
0,0,S6546,0,4,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,S6570,0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,S6570,0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,S6574,0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,S6574,0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,1996,S0916,2,18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1997,1997,S1262,2,14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1998,1998,S0404,2,11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
all_subjects.to_csv("Data/subjects.csv")
# all_subjects = pd.read_csv("Data/subjects.csv").drop(columns="Unnamed: 0")
# all_subjects